#### Data class

In [34]:
import random
class Sentiment:
    NEGATIVE ="NEGATIVE"
    POSITIVE = "POSITIVE"
    NEUTRAL = "NEUTRAL"

class Review:
    
    def __init__(self,text,score):     

        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        
        if self.score <=2:
            return Sentiment.NEGATIVE
        elif self.score ==3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE      
        
class ReviewContainer:
    
    def __init__(self,reviews):
        self.reviews=reviews
        
    def get_text(self):
        
        return [x.text for x in self.reviews]
        
    def get_sentiment(self):
        
        return [x.sentiment for x in self.reviews]
    
    
    def evenly_distribute(self):
        negative=list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE,self.reviews))
        positive=list(filter(lambda x: x.sentiment == Sentiment.POSITIVE,self.reviews))
        positive_shrunk=positive[0:len(negative)]
        self.reviews= negative + positive_shrunk
    
        random.shuffle(self.reviews)
        

#### Load Data

In [19]:
import json

file_name = 'books_small_10000.json' 

reviews = []
with open(file_name) as f:
    for line in f:
        review= json.loads(line) #it makes it a dictionary?
        reviews.append(Review(review['reviewText'],review['overall']))#appending objects!
                       
print(reviews[5].text)
print(reviews[5].sentiment)

I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia's trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character's voice on a strong subject and making it so that other peoples story may be heard through Mia's.
POSITIVE


#### Prep Data

In [36]:
from sklearn.model_selection import train_test_split
import numpy as np

#A=np.array([[1,2],[2,3],[4,5]])
#y=np.array([1,2,3])
#z=train_test_split(A,test_size=0.33,random_state=42)
#print(z) #this splits the matrix A in matrix [[2,3],[4,5]]] and matrix [[1,2]]

training,test=train_test_split(reviews,test_size=0.33,random_state=42)#both training and test are made of objects, reviews!

#print(training[0].text)# so training has both text and sentiment, the same for test, so we want to split them i two!
#print(training[0].sentiment)
#print(training[0].score)#we do not care about the score now?

train_container=ReviewContainer(training)
test_container = ReviewContainer(test)

len(cont.reviews)

872

In [53]:
train_container.evenly_distribute()
train_x = train_container.get_text()  #list comprehension!! works fine when training is a list of objects!!
train_y = train_container.get_sentiment()  #we split up training in x and y part, text and sentiment part.

test_container.evenly_distribute()
test_x = test_container.get_text()  #we split test in text 
test_y = test_container.get_sentiment()  # and sentiments!


print(train_y.count(Sentiment.NEGATIVE))
print(train_y.count(Sentiment.POSITIVE))

436
436


#### Bags of words vectorization

In [67]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer=TfidfVectorizer()

train_x_vectors = vectorizer.fit_transform(train_x)#here we basically make all sentences to vectors of numbers 
#print(train_x[0]) # this is just a sentence
#print(train_x_vectors[0]) this is the same sentence but expresed by numbers that count the repetition of words! in a long list.

test_x_vectors=vectorizer.transform(test_x)#we do not want to fit again we want to transform because it is out training data!!

#train_x_vectors
#train_y

#### Classification

#### Linear SVM

In [68]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear') #we create a classifier, need to learn the theory behind certain classifiers!
clf_svm.fit(train_x_vectors,train_y) #

clf_svm.predict(test_x_vectors[0])

print(test_x[0])#this sentence has been made
print(test_x_vectors[0])# a vector of counted words
print(clf_svm.predict(test_x_vectors[0]))#and here it has been predicted to be positive
print(test_y[0])#yes indeed it was positive!



I started it but I couldn't get locked into the story line at all of this book. Not for lack of trying.
  (0, 8213)	0.3195000857188416
  (0, 7976)	0.09569119762724264
  (0, 7929)	0.08533133688968354
  (0, 7533)	0.14955227671426183
  (0, 7463)	0.2970098708273701
  (0, 5478)	0.2076805940100372
  (0, 5408)	0.131885796561716
  (0, 4745)	0.47633609168016294
  (0, 4701)	0.2970098708273701
  (0, 4519)	0.3614557937659156
  (0, 4277)	0.1038402970050186
  (0, 4212)	0.2046197878045942
  (0, 3374)	0.19615381162937975
  (0, 3177)	0.12326679962166166
  (0, 1791)	0.26742794317114443
  (0, 1168)	0.13037507049005076
  (0, 991)	0.10487855738177373
  (0, 611)	0.16976158604870822
  (0, 347)	0.1730821448685445
['NEGATIVE']
NEGATIVE


#### Decision tree

In [69]:
from sklearn.tree import DecisionTreeClassifier 

clf_dec = DecisionTreeClassifier() #here we make a decision tree classifier

clf_dec.fit(train_x_vectors,train_y)

clf_dec.predict(test_x_vectors[0])#this classifier says positive too! where we know that test_y[0]=positive!

array(['NEGATIVE'], dtype='<U8')

#### Naive Bayes

#### Logistic Regression

#### Evaluation

In [70]:
clf_svm.score(test_x_vectors,test_y) #Here we use the trained model to evaluate it on the test part of the data!!!

0.8076923076923077

In [71]:
from sklearn.metrics import f1_score

print(f1_score(test_y, clf_svm.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))

[0.80582524 0.80952381]


In [72]:
test_set=['this book is nice','I did not like it that much','this book is weird','this book was interesting in the begining but turned to be boring in the end','the begining was horrible but it got better with time so I still recommend this book, because it had a good ending']

new_test=vectorizer.transform(test_set) #this transforms the test set into vectors of numbers!!!

clf_svm.predict(new_test)


array(['POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE'],
      dtype='<U8')

#### Tunning our model (with Grid Seach)

In [74]:
from sklearn.model_selection import GridSearchCV

tuned_svm = svm.SVC()
parameters ={'kernel':('linear','rbf'),'C':(1,2,3,4,5,32)}

clf=GridSearchCV(svm.SVC(),parameters,cv=5)
clf.fit(train_x_vectors,train_y)

C:\Users\Latitude\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Latitude\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Latitude\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Latitude\Anaconda3\lib\site-packages\sklearn\svm\base.py:193:

C:\Users\Latitude\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Latitude\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Latitude\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': (1, 2, 3, 4, 5, 32), 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [75]:
print(clf.score(test_x_vectors,test_y))

0.8076923076923077


#### Saving Model